# Convolution Neural Network

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-01-27 09:05:33.828695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.16.2'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#     tf.keras.layers.RandomBrightness(0.25, seed=10),
#     tf.keras.layers.RandomContrast(0.5, seed=20),
#     tf.keras.layers.RandomRotation(0.028, fill_mode="constant", seed=35),
# ])

train_datagen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
#train_datagen = data_augmentation
training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
#training_set = data_augmentation

Found 5216 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 624 images belonging to 2 classes.


## Part 2 - Builind the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 -Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64,3]))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 -Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 -Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 -Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 -Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics =['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test Set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 67s 408ms/step - accuracy: 0.7468 - loss: 0.5419 - val_accuracy: 0.7724 - val_loss: 0.4691
Epoch 2/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 401ms/step - accuracy: 0.8365 - loss: 0.3667 - val_accuracy: 0.7804 - val_loss: 0.4256
Epoch 3/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 399ms/step - accuracy: 0.8401 - loss: 0.3466 - val_accuracy: 0.8093 - val_loss: 0.3858
Epoch 4/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 64s 394ms/step - accuracy: 0.8726 - loss: 0.2974 - val_accuracy: 0.8157 - val_loss: 0.4060
Epoch 5/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 398ms/step - accuracy: 0.8681 - loss: 0.2957 - val_accuracy: 0.8686 - val_loss: 0.3295
Epoch 6/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 398ms/step - accuracy: 0.8996 - loss: 0.2496 - val_accuracy: 0.8846 - val_loss: 0.3035
Epoch 7/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 398ms/step - accuracy: 0.8886 - loss: 0.2574 - val_accuracy: 0.8574 - val_loss: 0.3186
Epoch 8/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 66s 406ms/step - accuracy: 0.8995 - loss: 0

# Part 4 -Making a single prediction

In [14]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'pneumonia'
else:
  prediction = 'not pneumonia'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


In [15]:
print(prediction)

not pneumonia
